In [1]:
import cv2
import time 
import matplotlib.pyplot as plt
import numpy as np

In [2]:
from keras.applications.resnet50 import ResNet50
from keras.optimizers import Adam
from keras.layers import *
from keras.models import Model

In [3]:
model = ResNet50(include_top=False, weights='imagenet', input_shape=(224, 224, 3))

In [4]:
av1 = GlobalAveragePooling2D()(model.output)
fc1 = Dense(256, activation='relu')(av1)
d1 = Dropout(0.5)(fc1)
fc2 = Dense(3, activation='softmax')(d1)

In [5]:
model_new = Model(inputs= model.input, outputs = fc2)
for ix in range(169):
    model_new.layers[ix].trainable = False
adam = Adam(learning_rate=0.00003)
model_new.compile(
    loss= 'categorical_crossentropy',
    optimizer=adam,
    metrics=['accuracy']
)

## Loading the weights

In [7]:
model_new.load_weights("./mob_model_new.h5")

## Demo for set-up

In [8]:
cap = cv2.VideoCapture(0)

TIMER = int(3) 

while(True):
    
    ret, frame = cap.read()
    if ret == False:
        continue
    
    cv2.rectangle(frame,(50,100),(250,350),(255,0,0),2)
    cv2.rectangle(frame,(400,100),(600,350),(0,255,0),2)
    
    frame = cv2.resize(frame, (1000, 700))
    
    frame = cv2.flip(frame,1)
    cv2.imshow("Rock Paper Scissors", frame)
    
    key_pressed = cv2.waitKey(1) & 0xFF
    
    if key_pressed == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

In [9]:
labels_dict = {
    "Paper" : 0,
    "Rock" : 1,
    "Scissor" : 2
}
rev_label_dict = dict([(value, key) for (key, value) in labels_dict.items()])

## Winner function for scoring

In [10]:
def winner(f1, f2):
    
    p1 = rev_label_dict.get((np.argmax(model_new.predict(f1.reshape(1,224,224,3)))))
    p2 = rev_label_dict.get((np.argmax(model_new.predict(f2.reshape(1,224,224,3)))))
    
    if (p1 == "Rock" and p2 =="Scissor") or (p1 == "Paper" and p2 == "Rock") or ( p1 == "Scissor" and p2 == "Paper"):
        return 1
    
    elif (p2 == "Rock" and p1 =="Scissor") or (p2 == "Paper" and p1 == "Rock") or ( p2 == "Scissor" and p1 == "Paper"):
        return -1
    
    return 0

## Merging WebCam feed with the model
## Continuous Scoring
## Best of 5

In [11]:
cap = cv2.VideoCapture(0)
score1 = 0
score2 = 0
TIMER = int(3) 
count = 0

cv2.waitKey(125)

while(True):
    
    ret, frame = cap.read()
    if ret == False:
        continue
    
    cv2.rectangle(frame,(50,100),(274,324),(255,0,0),2)
    cv2.rectangle(frame,(400,100),(624,324),(255,0,0),2)
    cv2.putText(frame , str(score1) , (50,570) , cv2.FONT_HERSHEY_SIMPLEX , 4 ,(0,69,255) , 2 , cv2.LINE_AA )
    cv2.putText(frame , str(score2) , (600,570) , cv2.FONT_HERSHEY_SIMPLEX , 4 ,(0,69,255) , 2 , cv2.LINE_AA )
    
    frame = cv2.resize(frame, (1000, 700))
    
    frame = cv2.flip(frame,1)
    cv2.imshow("Rock Paper Scissors", frame)
    
    prev = time.time()
    
    while TIMER >= 0:
        
        ret, frame = cap.read()
        if ret == False:
            continue

        cv2.rectangle(frame,(50,100),(274,324),(255,0,0),2)
        cv2.rectangle(frame,(400,100),(624,324),(255,0,0),2)

        frame = cv2.resize(frame, (1000, 700))

        frame = cv2.flip(frame,1)
        
        font = cv2.FONT_HERSHEY_SIMPLEX 
        cv2.putText(frame, str(TIMER),  
                    (400, 250), font, 
                    7, (0, 0, 255), 
                    4, cv2.LINE_AA)
        cv2.putText(frame , str(score1) , (50,570) , cv2.FONT_HERSHEY_SIMPLEX , 4 ,(0,69,255) , 2 , cv2.LINE_AA )
        cv2.putText(frame , str(score2) , (600,570) , cv2.FONT_HERSHEY_SIMPLEX , 4 ,(0,69,255) , 2 , cv2.LINE_AA )
        
        cv2.imshow("Rock Paper Scissors", frame)
        cv2.waitKey(125)
        cur = time.time()
        
        if cur-prev >=1:
            prev = cur
            TIMER = TIMER - 1
            
    else:
        
        ret, frame = cap.read()
        if ret == False:
            continue
        
        
        # Images for the model
        f1 = frame[100:324, 400:624]
        f2 = frame[100:324, 50:274]
        
        if winner(f1, f2) == 1:
            cv2.rectangle(frame,(50,100),(274,324),(255,0,0),2)
            cv2.rectangle(frame,(400,100),(624,324),(0,255,0),2)
            frame = cv2.resize(frame, (1000, 700))
            frame = cv2.flip(frame,1)
            score1 += 1
            cv2.putText(frame , "Won" , (50,95) , cv2.FONT_HERSHEY_SIMPLEX , 4 ,(0,69,255) , 2 , cv2.LINE_AA )
            cv2.putText(frame , str(score1) , (50,570) , cv2.FONT_HERSHEY_SIMPLEX , 4 ,(0,69,255) , 2 , cv2.LINE_AA )
            cv2.putText(frame , str(score2) , (600,570) , cv2.FONT_HERSHEY_SIMPLEX , 4 ,(0,69,255) , 2 , cv2.LINE_AA )
        elif winner(f1, f2) == -1:
            cv2.rectangle(frame,(50,100),(274,324),(0,255,0),2)
            cv2.rectangle(frame,(400,100),(624,324),(255,0,0),2)
            frame = cv2.resize(frame, (1000, 700))
            frame = cv2.flip(frame,1)
            score2 += 1
            cv2.putText(frame , "Won" , (600,95) , cv2.FONT_HERSHEY_SIMPLEX , 4 ,(0,69,255) , 2 , cv2.LINE_AA )
            cv2.putText(frame , str(score1) , (50,570) , cv2.FONT_HERSHEY_SIMPLEX , 4 ,(0,69,255) , 2 , cv2.LINE_AA )
            cv2.putText(frame , str(score2) , (600,570) , cv2.FONT_HERSHEY_SIMPLEX , 4 ,(0,69,255) , 2 , cv2.LINE_AA )
        else:
            cv2.rectangle(frame,(50,100),(274,324),(0,255,0),2)
            cv2.rectangle(frame,(400,100),(624,324),(0,255,0),2)
            frame = cv2.resize(frame, (1000, 700))
            frame = cv2.flip(frame,1)
            cv2.putText(frame , "Draw" , (50,95) , cv2.FONT_HERSHEY_SIMPLEX , 4 ,(0,69,255) , 2 , cv2.LINE_AA )
            cv2.putText(frame , "Draw" , (600,95) , cv2.FONT_HERSHEY_SIMPLEX , 4 ,(0,69,255) , 2 , cv2.LINE_AA )
            cv2.putText(frame , str(score1) , (50,570) , cv2.FONT_HERSHEY_SIMPLEX , 4 ,(0,69,255) , 2 , cv2.LINE_AA )
            cv2.putText(frame , str(score2) , (600,570) , cv2.FONT_HERSHEY_SIMPLEX , 4 ,(0,69,255) , 2 , cv2.LINE_AA )
            
        cv2.imshow("Rock Paper Scissors", frame)
        cv2.waitKey(2000)
        
        TIMER = int(3)
        count += 1
        
    if count == 5:
        break
        
    key_pressed = cv2.waitKey(1) & 0xFF
    
    if key_pressed == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()